In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
from surprise import Reader, Dataset, SVD, evaluate,accuracy
from surprise.model_selection import train_test_split

In [3]:
all_ratings_DF =  pd.read_csv('all_ratings.csv')

In [4]:
all_ratings_DF.head(5)

,movie_id,user_id,rating
0,4500,2532865,4
1,4500,573364,3
2,4500,1696725,3
3,4500,1253431,3
4,4500,1265574,2


In [5]:
all_ratings_DF.shape

(100480507, 3)

In [6]:
all_ratings_DF2 = all_ratings_DF

In [7]:
f = ['count','mean']

ratings_summary_DF = all_ratings_DF.groupby('movie_id')['rating'].agg(f)
ratings_summary_DF.index = ratings_summary_DF.index.map(int)

movie_benchmark = round(ratings_summary_DF['count'].quantile(0.8),0)
drop_movie_list = ratings_summary_DF[ratings_summary_DF['count'] < movie_benchmark].index

print('Movie minimum times of review: {}'.format(movie_benchmark))


Movie minimum times of review: 4040.0


In [8]:
user_summary_DF = all_ratings_DF.groupby('user_id')['rating'].agg(f)
user_summary_DF.index = user_summary_DF.index.map(int)

cust_benchmark = round(user_summary_DF['count'].quantile(0.8),0)
drop_cust_list = user_summary_DF[user_summary_DF['count'] < cust_benchmark].index

print('Customer minimum times of review: {}'.format(cust_benchmark))

Customer minimum times of review: 322.0


In [9]:
print('Original Shape: {}'.format(all_ratings_DF.shape))
all_ratings_DF = all_ratings_DF[~all_ratings_DF['movie_id'].isin(drop_movie_list)]
all_ratings_DF = all_ratings_DF[~all_ratings_DF['user_id'].isin(drop_cust_list)]
print('After Trim Shape: {}'.format(all_ratings_DF.shape))
print('-Data Examples-')
print(all_ratings_DF.iloc[::5000000, :])

Original Shape: (100480507, 3)
After Trim Shape: (56222526, 3)
-Data Examples-
          movie_id  user_id  rating
3151          4506   716091       4
8845673       5971  1498163       3
17841467      7399   238673       4
26508188      9125  1707006       3
35307989     10920   480305       5
44425386     12470   186951       3
53551901       758  2404366       4
62517676      2457  2647259       4
71328360      4109   632719       4
80290897     14480  1103969       3
89141979     15844  2535126       3
98305686     17355  2381067       3


In [10]:
all_ratings_DF_PVT = pd.pivot_table(all_ratings_DF,values='rating',index='user_id',columns='movie_id')

print(all_ratings_DF_PVT.shape)
all_ratings_DF_PVT.head(5)


(96290, 3554)


movie_id,8,17,18,26,28,30,33,44,46,52,...,17714,17724,17725,17730,17743,17756,17761,17762,17764,17769
user_id,,,,,,,,,,,,,,,,,,,,,
6,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,5.0,NaN,NaN,NaN,4.0,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN
79,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN


In [11]:
import csv

all_movies_DF = pd.DataFrame(columns=['movie_id','year','movie_name'])

csv_file = open('movie_titles.csv',encoding = "ISO-8859-1")
csv_reader = csv.reader(csv_file)
for row in csv_reader:
    if(len(row) == 3):
        all_movies_DF.loc[row[0]] = [int(row[0]),row[1],row[2]]
    else:
        temp_movie_title = ''
        for j in (2,len(row)-1):
            if (j==len(row)-1):
                temp_movie_title = temp_movie_title + row[j]
            else:
                temp_movie_title = temp_movie_title + row[j] + ','
        all_movies_DF.loc[row[0]] = [int(row[0]),row[1],temp_movie_title]

csv_file.close()

In [12]:
all_movies_DF.head(5)

,movie_id,year,movie_name
1,1,2003,Dinosaur Planet
2,2,2004,Isle of Man TT 2004 Review
3,3,1997,Character
4,4,1994,Paula Abdul's Get Up & Dance
5,5,2004,The Rise and Fall of ECW


In [17]:
reader = Reader()
readerS = Reader(rating_scale=(1, 5))
svd = SVD()

In [18]:
def PC_Evaluate():
    train_movie_data = Dataset.load_from_df(all_ratings_DF[['user_id', 'movie_id', 'rating']][:100000], reader)
    train_movie_data.split(n_folds=3)
    svd = SVD()
    evaluate(svd, train_movie_data, measures=['RMSE', 'MAE'])

PC_Evaluate()

/anaconda3/envs/pythonData1/lib/python3.6/site-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/anaconda3/envs/pythonData1/lib/python3.6/site-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 0.9853
MAE:  0.7770
------------
Fold 2
RMSE: 0.9887
MAE:  0.7804
------------
Fold 3
RMSE: 0.9919
MAE:  0.7818
------------
------------
Mean RMSE: 0.9887
Mean MAE : 0.7797
------------
------------


In [24]:

def PC_TrainTest():
    reader = Reader()
    readerS = Reader(rating_scale=(1, 5))
    svd = SVD()
    all_ratings_data = Dataset.load_from_df(all_ratings_DF[['user_id', 'movie_id', 'rating']][:100000], reader)
    all_ratings_train_data, all_ratings_test_data = train_test_split(all_ratings_data, test_size=.25)

          
    svd.fit(all_ratings_train_data)
    predictions = svd.test(all_ratings_test_data)
    RMSE = accuracy.rmse(predictions)
    MAE = accuracy.mae(predictions)
    print('RMSE : ' ,RMSE)
    print('MAE : ' ,MAE)
    predictions_DF = pd.DataFrame(predictions)
    predictions_DF = predictions_DF.rename(columns={"uid": "user_id"
                               , "iid": "movie_id"
                               , "r_ui": "rating"
                               , "est": "prediction"
                              })
    predictions_DF.to_csv('movie_predictions_SVD.csv', index=True, header=True)


PC_TrainTest()

RMSE: 0.9892
MAE:  0.7789
RMSE :  0.9891856063136696
MAE :  0.7788761348011874


In [ ]:
## This step is till WIP
def PC_Prediction(user):
    data = Dataset.load_from_df(user, reader)
    trainset = data.build_full_trainset()
    svd.train(trainset)
    user_785314['Estimate_Score'] = user_785314['Movie_Id'].apply(lambda x: svd.predict(785314, x).est)

In [ ]:
user = all_ratings_DF[(all_ratings_DF['Cust_Id'] == 785314) & (all_ratings_DF['Rating'] == 5)]